In [1]:
X=[]
Z=[]
IMG_SIZE=150
PAR_DIR = 'C:\\Users\\LAKSHAY SINGHAL\\Desktop\\withParkinsonDraw'
NOPAR_DIR = 'C:\\Users\\LAKSHAY SINGHAL\\Desktop\\withoutParkinsonDraw'

In [2]:
from tqdm import tqdm
import os      
import cv2
import numpy as np
from skimage.filters import threshold_otsu
from skimage import color
from skimage.feature import canny

def assign_label(img,wtype):
    return wtype

def make_train_data(wtype,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,wtype)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        imgarr = np.array(img)
        imgB = color.rgb2gray(imgarr)
        thresh = threshold_otsu(imgB)
        binary1 = imgB < thresh
        canny_edges = canny(imgB, sigma=0)
        
        X.append(canny_edges)
        Z.append(str(label))
        
make_train_data('withParkinson',PAR_DIR)
make_train_data('withoutParkinson',NOPAR_DIR)

100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:01<00:00, 49.61it/s]


In [3]:
for i in range(160):
    X[i] = np.expand_dims(X[i], axis=3)

c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  


In [4]:
X[0].shape

(150, 150, 1)

In [5]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical

Using TensorFlow backend.


In [6]:
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,2)
X=np.array(X)

In [7]:
X=X/255
X.shape

(160, 150, 150, 1)

In [8]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [9]:
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(150, 150, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(100, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(2, activation='softmax'))

W1107 22:59:11.259068 14876 deprecation_wrapper.py:119] From c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



In [10]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
model.fit(x_train,y_train,epochs=40, validation_data=(x_test,y_test))

W1107 22:59:17.138548 14876 deprecation_wrapper.py:119] From c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Train on 120 samples, validate on 40 samples
Epoch 1/40
120/120 [==============================] - ETA: 1s - loss: 0.6933 - accuracy: 0.53 - ETA: 0s - loss: 0.6938 - accuracy: 0.50 - ETA: 0s - loss: 0.6763 - accuracy: 0.61 - 2s 18ms/step - loss: 0.6583 - accuracy: 0.6333 - val_loss: 0.6363 - val_accuracy: 0.5750
Epoch 2/40
120/120 [==============================] - ETA: 1s - loss: 0.5057 - accuracy: 0.68 - ETA: 0s - loss: 0.6090 - accuracy: 0.59 - ETA: 0s - loss: 0.5866 - accuracy: 0.58 - 2s 15ms/step - loss: 0.5595 - accuracy: 0.6333 - val_loss: 0.5693 - val_accuracy: 0.6750
Epoch 3/40
120/120 [==============================] - ETA: 1s - loss: 0.4107 - accuracy: 0.87 - ETA: 0s - loss: 0.4228 - accuracy: 0.90 - ETA: 0s - loss: 0.4267 - accuracy: 0.92 - 2s 15ms/step - loss: 0.4295 - accuracy: 0.9167 - val_loss: 0.5515 - val_accuracy: 0.7000
Epoch 4/40
120/120 [==============================] - ETA: 1s - loss: 0.4111 - accuracy: 0.84 - ETA: 0s - loss: 0.3769 - accuracy: 0.89 - ETA: 0s - 

In [43]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import lite

In [44]:
keras_file = "parkinson5.h5"
keras.models.save_model(model, keras_file)

In [45]:
converter = lite.TocoConverter.from_keras_model_file(keras_file)
tflite_model = converter.convert()
open("parkinson5.tflite", "wb").write(tflite_model)

W1107 23:43:41.432426 14876 deprecation.py:323] From <ipython-input-45-0801713ffa87>:1: TocoConverter.from_keras_model_file (from tensorflow.lite.python.lite) is deprecated and will be removed in a future version.
Instructions for updating:
Use `lite.TFLiteConverter.from_keras_model_file` instead.
W1107 23:43:44.446586 14876 deprecation.py:323] From c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\tensorflow\lite\python\util.py:238: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W1107 23:43:44.448196 14876 deprecation.py:323] From c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\graph_util_impl.py:270: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future ver

70096912

In [42]:
correct=[]
wrong=[]
for img in tqdm(os.listdir(PAR_DIR)):
    path = os.path.join(PAR_DIR,img)
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    imgarr= np.array(img)
    imgB = color.rgb2gray(imgarr)
    thresh = threshold_otsu(imgB)
    binary1 = imgB < thresh
    canny_edges = canny(imgB, sigma=0)
    X = np.expand_dims(canny_edges, axis=3)
    X.resize(1,150,150,1)
    pred1=model.predict(X)
    print(pred1)
    print(np.argmax(pred1))
    if(np.argmax(pred1)==0):
        correct.append(np.argmax(pred1))
    if(not np.argmax(pred1)==0):
        wrong.append(np.argmax(pred1))
print(len(correct))
print(len(wrong))

  0%|                                                                                           | 0/93 [00:00<?, ?it/s]c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  if sys.path[0] == '':


[[1. 0.]]
0
[[1. 0.]]
0
[[1. 0.]]
0
[[1. 0.]]
0


  4%|███▌                                                                               | 4/93 [00:00<00:02, 35.49it/s]

[[1. 0.]]
0
[[1. 0.]]
0
[[1. 0.]]
0
[[1. 0.]]
0


  9%|███████▏                                                                           | 8/93 [00:00<00:02, 35.71it/s]

...

[[1. 0.]]
0
[[1. 0.]]
0
[[1. 0.]]
0


 97%|███████████████████████████████████████████████████████████████████████████████▎  | 90/93 [00:03<00:00, 26.02it/s]

[[1. 0.]]
0
[[1. 0.]]
0
[[1. 0.]]
0


100%|██████████████████████████████████████████████████████████████████████████████████| 93/93 [00:03<00:00, 25.53it/s]


93
0


In [41]:
correct=[]
wrong=[]
for img in tqdm(os.listdir(NOPAR_DIR)):
    path = os.path.join(NOPAR_DIR,img)
    img = cv2.imread(path,cv2.IMREAD_COLOR)
    img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
    imgarr= np.array(img)
    imgB = color.rgb2gray(imgarr)
    thresh = threshold_otsu(imgB)
    binary1 = imgB < thresh
    canny_edges = canny(imgB, sigma=0)
    X = np.expand_dims(canny_edges, axis=3)
    X.resize(1,150,150,1)
    pred1=model.predict(X)
    print(pred1)
    print(np.argmax(pred1))
    if(np.argmax(pred1)==1):
        correct.append(np.argmax(pred1))
    if(not np.argmax(pred1)==1):
        wrong.append(np.argmax(pred1))
print(len(correct))
print(len(wrong))

  0%|                                                                                           | 0/67 [00:00<?, ?it/s]c:\users\lakshay singhal\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.
  if sys.path[0] == '':


[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1


  4%|███▋                                                                               | 3/67 [00:00<00:02, 28.17it/s]

[[0. 1.]]
1
[[0. 1.]]
1
[[1.0000000e+00 2.9960527e-34]]
0


  9%|███████▍                                                                           | 6/67 [00:00<00:02, 26.83it/s]

...

[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1


 96%|██████████████████████████████████████████████████████████████████████████████▎   | 64/67 [00:02<00:00, 23.82it/s]

[[0. 1.]]
1
[[0. 1.]]
1
[[0. 1.]]
1


100%|██████████████████████████████████████████████████████████████████████████████████| 67/67 [00:02<00:00, 24.24it/s]


58
9
